In [1]:
import numpy as np
import pandas as pd
from dask import dataframe as dd
import string
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = nlp.tokenizer
stemmer = SnowballStemmer("english")

pd.set_option('display.max_columns', 100)

# Government Spend Analysis

## Step 1: Examine column headers to identify columns of interest

In [2]:
test = pd.read_csv('FY2016_All_Contracts_Full_20210608_1.csv', nrows = 5)

test

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,...,manufacturer_of_goods,veterinary_hospital,hispanic_servicing_institution,receives_contracts,receives_financial_assistance,receives_contracts_and_financial_assistance,airport_authority,council_of_governments,housing_authorities_public_tribal,interstate_entity,planning_commission,port_authority,transit_authority,subchapter_scorporation,limited_liability_corporation,foreign_owned,for_profit_organization,nonprofit_organization,other_not_for_profit_organization,the_ability_one_program,private_university_or_college,state_controlled_institution_of_higher_learning,1862_land_grant_college,1890_land_grant_college,1994_land_grant_college,minority_institution,historically_black_college,tribal_college,alaskan_native_servicing_institution,native_hawaiian_servicing_institution,school_of_forestry,veterinary_college,dot_certified_disadvantage,self_certified_small_disadvantaged_business,small_disadvantaged_business,c8a_program_participant,historically_underutilized_business_zone_hubzone_firm,sba_certified_8a_joint_venture,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,1,0,0,8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",DU205NC16D02,0,799750.00,546621.23,799750.00,546621.23,799750.00,546621.23,NaN,NaN,NaN,2016-09-30,2016,2016-09-30 00:00:00,2021-09-29 00:00:00,2021-09-29 00:00:00,NaN,NaN,86,DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT (HUD),8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",86545A,DEPT HUD-CHIEF PROCUREMENT OFFICER,86,DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT (HUD),8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",8630AX,DEPT OF HOUSING URBAN DEVELOPMENT,086-2016/2017-0183-000,086-0183,25.3: Other goods and services from Federal so...,0709: ADMINISTRATIVE EXPENSES,X,NOT APPLICABLE,NaN,NaN,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,NaN,831854596,...,f,f,f,f,f,t,f,f,f,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,t,f,t,f,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_000...,2019-02-12 12:16:44
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,1,5,0,6800,ENVIRONMENTAL PROTECTION AGENCY,EPR91502,0,0.00,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,2016-09-30,2016,2016-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,NaN,NaN,68,ENVIRO

### Columns of interest:

* contract_transaction_unique_key
* contract_award_unique_key
* awarding_agency_code
* awarding_sub_agency_code
* awarding_office_code
* funding_agency_code
* funding_sub_agency_code
* funding_office_code
* recipient_duns
* recipient_name
* recipient_doing_business_as_name
* recipient_parent_duns


## Step 2: Load data

Using dask with only columns of interest included.

In [3]:
columns = ['contract_transaction_unique_key', 'contract_award_unique_key', 'awarding_agency_code', 'awarding_sub_agency_code',\
           'awarding_office_code', 'funding_agency_code', 'funding_sub_agency_code', 'funding_office_code', 'recipient_duns', \
           'recipient_name', 'recipient_doing_business_as_name', 'recipient_parent_duns']

data = dd.read_csv('*.csv', usecols=columns, dtype={'awarding_sub_agency_code': 'object', 'funding_sub_agency_code': 'object', \
                                                    'recipient_doing_business_as_name': 'object', 'recipient_parent_duns': 'object',\
                                                    'funding_agency_code': 'object', 'awarding_office_code': 'object', \
                                                    'funding_office_code': 'object', 'recipient_duns': 'object'})

data.head()

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,86,8600,86545A,086,8600,8630AX,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,831854596
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987
2,6800_6800_0001_20_EPR51502_0,CONT_AWD_0001_6800_EPR51502_6800,68,6800,68HE05,068,6800,68U000,079493355,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,NaN,079493355
3,6800_6800_0001_0_EPS21602_0,CONT_AWD_0001_6800_EPS21602_6800,68,6800,68HE02,068,6800,68R000,080265258,CAT-ENSOFTEK,NaN,080265258
4,6800_6800_0001_31_EPS51301_0,CONT_AWD_0001_6800_EPS51301_6800,68,6800,68HE05,068,6800,68U000,178599221,"TETRA TECH EM, INC.",NaN,045224250


### Step 2a: Save as parquet format

(So it's easier to reload if needed)

In [4]:
data.to_parquet('government_spend_dataset.parquet', engine='pyarrow')

(None,)

## Step 3: Get number of initial records

In [5]:
a = data.index.size.compute()

print("There are ", a, " initial records in this dataset.")

There are  4820290  initial records in this dataset.


## Step 4: Create list of unique suppliers (exact)

(and get number of unique suppliers)

In [6]:
suppliers_unique = data.recipient_name.unique().compute().reset_index()

print("There are ", len(suppliers_unique), " unique suppliers (exact match only).")

There are  153462  unique suppliers (exact match only).


In [7]:
suppliers_unique.head()

,index,recipient_name
0,0,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC"
1,1,INNOVATE INC
2,2,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...
3,3,CAT-ENSOFTEK
4,4,"TETRA TECH EM, INC."


### Step 4a: Does recipient_doing_business_as_name have any non-null values?

Answer: yes, hold off on this until next step

In [8]:
data.recipient_doing_business_as_name.dropna().compute()

23                MIT LINCOLN LABORATORY
29                                 U T C
39        SOFTWARE ENGINEERING INSTITUTE
52                    PACIFIC SCIENTIFIC
86                    PACIFIC SCIENTIFIC
                      ...               
18863    TOUCHSTONE NEURORECOVERY CENTER
18864                     PROVIDE A RIDE
18875             PETNET PHARMACEUTICALS
18882        MUSC DEPARTMENT OF MEDICINE
18895                 BIMBO BAKERIES USA
Name: recipient_doing_business_as_name, Length: 94562, dtype: object

## Step 5: Normalize text

Using:

 - list of punctuation from string package
 - custom regex to remove spaces between single character words
 - spacy English tokenizer
 - Snowball stemmer from NLTK
 - English stopwords from NLTK (updated to not use stopwords list)

Returning normalized tokens as list in additional column

In [10]:
# stop = set(stopwords.words('english'))
punct = string.punctuation

def remove_punct(text):
    text_str = str(text)
    for c in text_str:
        if c in punct:
            text_str = text_str.replace(c, '')
    return text_str

def remove_whitespace(text_str):
    text_str = re.sub(r'(?<=\s[A-Z0-9])[ ](?=[A-Z0-9]\s)', '', text_str)
    text_str = re.sub(r'(?<=^[A-Z0-9])[ ](?=[A-Z0-9])', '', text_str)
    text_str = re.sub(r'(?<=[A-Z0-9])[ ](?=[A-Z0-9]$)', '', text_str)
    return text_str

def normalize_text(text_str):
    wordlist = []
    for w in tokenizer(text_str):
        if not w.is_punct:
            wordlist.append(stemmer.stem(str(w).lower()))
    return wordlist

suppliers_unique['recipient_name_norm'] = suppliers_unique.recipient_name.apply(remove_punct).apply(remove_whitespace).apply(normalize_text)

suppliers_unique

,index,recipient_name,recipient_name_norm
0,0,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC","[mortgag, assist, and, acquisit, group, llc]"
1,1,INNOVATE INC,"[innov, inc]"
2,2,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,"[ryba, marin, construct, coterra, contract, se..."
3,3,CAT-ENSOFTEK,[catensoftek]
4,4,"TETRA TECH EM, INC.","[tetra, tech, em, inc]"
...,...,...,...
153457,153457,"AZORMED - GESTAO AMBIENTAL ACOREANA, LDA","[azorm, , gestao, ambient, acoreana, lda]"
153458,153458,JEAN ROB & CO,"[jean, rob, , co]"
153459,153459,"WOODS SERVICES, INC.","[wood, servic, inc]"
153460,153460,FLOWERS & ASSOCIATES,"[flower, , associ]"


In [11]:
suppliers_unique['recipient_name_str'] = suppliers_unique.recipient_name_norm.agg(lambda x: ','.join(map(str, x)))

suppliers_unique.recipient_name_str.value_counts()[suppliers_unique.recipient_name_str.value_counts() > 1].\
sort_values(ascending = False).to_frame(name = 'Additional matches')

,Additional matches
"innov,technolog,inc",5
"analyt,servic,inc",5
"macro,intern,inc",4
"shredit,usa,inc",4
"epsilon,system,solut,inc",4
...,...
"angarai,intern,inc",2
"total,resourc,manag,inc",2
"hawk,enterpris,llc",2
"sbiw,inc",2


There were 3,734 additional matches discovered through this simple text normalization.

In [14]:
suppliers_unique.recipient_name_str.unique().shape

(149545,)

With normalized text, **149,545** unique suppliers are found. These new normalized names can be joined to the original df.

In [16]:
data_norm = data.merge(suppliers_unique, on = 'recipient_name').compute()

data_norm

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns,index,recipient_name_norm,recipient_name_str
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,86,8600,86545A,086,8600,8630AX,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,831854596,0,"[mortgag, assist, and, acquisit, group, llc]","mortgag,assist,and,acquisit,group,llc"
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987,1,"[innov, inc]","innov,inc"
2,6800_6800_0002_3_EPR91502_0,CONT_AWD_0002_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987,1,"[innov, inc]","innov,inc"
3,6800_6800_0001_20_EPR51502_0,CONT_AWD_0001_6800_EPR51502_6800,68,6800,68HE05,068,6800,68U000,079493355,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,NaN,079493355,2,"[ryba, marin, construct, coterra, contract, se...","ryba,marin,construct,coterra,contract,servic,l..."
4,6800_6800_0001_0_EPS21602_0,CONT_AWD_0001_6800_EPS21602_6800,68,6800,68HE02,068,6800,68R000,080265258,CAT-ENSOFTEK,NaN,080265258,3,[catensoftek],catensoftek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18914,7008_-NONE-_HSCG3216PSMA006_P00001_-NONE-_0,CONT_AWD_HSCG3216PSMA006_7008_-NONE-_-NONE-,70,7008,00032,070,7008,00032,785365693,"FLO-TECH, L.L.C.",NaN,785365693,27651,"[flotech, llc]","flotech,llc"
18915,9700_-NONE-_SPE4A716D5005_0_-NONE-_-NONE-,CONT_IDV_SPE4A716D5005_9700,97,97AS,SPE4A7,097,97AS,SPE4A7,089107726,"PRECI-MANUFACTURING, INCORPORATED",NaN,089107726,9467,"[precimanufactur, incorpor]","precimanufactur,incorpor"
18916,3600_-NONE-_VA25815P0001_P00002_-NONE-_0,CONT_AWD_VA25815P0001_3600_-NONE-_-NONE-,36,3600,00258,036,3600,00678,968237433,GARY BLUEMKE,NaN,968237433,153461,"[gari, bluemk]","gari,bluemk"
18917,9700_-NONE-_SPE4A716D0005_0_-NONE-_-NONE-,CONT_IDV_SPE4A716D0005_9700,97,97AS,SPE4A7,097,97AS,SPE4A7,121360627,"JANEL'S INDUSTRIES, INC.",NaN,121360627,996,"[janel, industri, inc]","janel,industri,inc"


### Step 5a: Normalizing and matching recipient_doing_business_as_name column

Names also appear for some rows in recipient_doing_business_as_name. For rows where this appears, this can also be normalized and matched to normed values in recipient_name column.

In [24]:
suppliers_unique_rev = data_norm.recipient_doing_business_as_name.dropna().drop_duplicates().reset_index()

suppliers_unique_rev

,index,recipient_doing_business_as_name
0,109,MIT LINCOLN LABORATORY
1,136,BOEING
2,158,U T C
3,224,SOFTWARE ENGINEERING INSTITUTE
4,238,INTERNATIONAL BUSINESS MACHINES
...,...,...
8893,18571,GOLDEN LIVINGCENTER - BRIARWOOD
8894,18756,AVALON PLACE WHARTON
8895,18758,BRIARWOOD NURSING&REHABILITATION
8896,18849,FAIRFIELD INN -MARRIOTT


In [26]:
suppliers_unique_rev['recipient_doing_business_norm'] = suppliers_unique_rev.recipient_doing_business_as_name.apply(remove_punct).\
apply(remove_whitespace).\
apply(normalize_text)

suppliers_unique_rev['recipient_doing_business_str'] = suppliers_unique_rev.recipient_doing_business_norm.agg(lambda x: ','.join(map(str, x)))

suppliers_unique_rev

,index,recipient_doing_business_as_name,recipient_doing_business_norm,recipient_doing_business_str
0,109,MIT LINCOLN LABORATORY,"[mit, lincoln, laboratori]","mit,lincoln,laboratori"
1,136,BOEING,[boe],boe
2,158,U T C,[utc],utc
3,224,SOFTWARE ENGINEERING INSTITUTE,"[softwar, engin, institut]","softwar,engin,institut"
4,238,INTERNATIONAL BUSINESS MACHINES,"[intern, busi, machin]","intern,busi,machin"
...,...,...,...,...
8893,18571,GOLDEN LIVINGCENTER - BRIARWOOD,"[golden, livingcent, , briarwood]","golden,livingcent, ,briarwood"
8894,18756,AVALON PLACE WHARTON,"[avalon, place, wharton]","avalon,place,wharton"
8895,18758,BRIARWOOD NURSING&REHABILITATION,"[briarwood, nursingrehabilit]","briarwood,nursingrehabilit"
8896,18849,FAIRFIELD INN -MARRIOTT,"[fairfield, inn, marriott]","fairfield,inn,marriott"


In [33]:
data_norm_rev = data_norm.merge(suppliers_unique_rev, how = 'outer', on = ['recipient_doing_business_as_name'])

data_norm_rev

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns,index_x,recipient_name_norm,recipient_name_str,index_y,recipient_doing_business_norm,recipient_doing_business_str
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,86,8600,86545A,086,8600,8630AX,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,831854596,0,"[mortgag, assist, and, acquisit, group, llc]","mortgag,assist,and,acquisit,group,llc",NaN,NaN,NaN
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987,1,"[innov, inc]","innov,inc",NaN,NaN,NaN
2,6800_6800_0002_3_EPR91502_0,CONT_AWD_0002_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987,1,"[innov, inc]","innov,inc",NaN,NaN,NaN
3,6800_6800_0001_20_EPR51502_0,CONT_AWD_0001_6800_EPR51502_6800,68,6800,68HE05,068,6800,68U000,079493355,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,NaN,079493355,2,"[ryba, marin, construct, coterra, contract, se...","ryba,marin,construct,coterra,contract,servic,l...",NaN,NaN,NaN
4,6800_6800_0001_0_EPS21602_0,CONT_AWD_0001_6800_EPS21602_6800,68,6800,68HE02,068,6800,68R000,080265258,CAT-ENSOFTEK,NaN,080265258,3,[catensoftek],catensoftek,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4820285,3600_-NONE-_VA24715A0007_P00001_-NONE-_-NONE-,CONT_IDV_VA24715A0007_3600,36,3600,00247,036,3600,00508,622596158,GGNSC TUCKER BRIARWOOD LLC,GOLDEN LIVINGCENTER - BRIARWOOD,621561971,153428,"[ggnsc, tucker, briarwood, llc]","ggnsc,tucker,briarwood,llc",18571.0,"[golden, livingcent, , briarwood]","golden,livingcent, ,briarwood"
4820286,3600_-NONE-_VA25615G0083_0_-NONE-_-NONE-,CONT_IDV_VA25615G0083_3600,36,3600,00256,036,3600,00580,079478766,"SOUTHWEST LTC-WHARTON, LTD.",AVALON PLACE WHARTON,079478766,153440,"[southwest, ltcwharton, ltd]","southwest,ltcwharton,ltd",18756.0,"[avalon, place, wharton]","avalon,place,wharton"
4820287,3600_-NONE-_VA25615A0093_0_-NONE-_-NONE-,CONT_IDV_VA25615A0093_3600,36,3600,00256,036,3600,00580,795449045,"PINNACLE HEALTH FACILITIES XV, LP",BRIARWOOD NURSING&REHABILITATION,795431076,122878,"[pinnacl, health, facil, xv, lp]","pinnacl,health,facil,xv,lp",18758.0,"[briarwood, nursingrehabilit]","briarwood,nursingrehabilit"
4820288,9700_-NONE-_N6830616A0073_0_-NONE-_-NONE-,CONT_IDV_N6830616A0073_9700,97,1700,N68306,097,1700,N68306,808502129,"OCEAN PROPERTIES, LTD.",FAIRFIELD INN -MARRIOTT,NaN,92253,"[ocean, properti, ltd]","ocean,properti,ltd",18849.0,"[fairfield, inn, marriott]","fairfield,inn,marriott"


In [39]:
data_norm_rev['recipient_name_str'] = np.where(data_norm_rev.recipient_doing_business_str.str in suppliers_unique.recipient_name_str, \
                                                 data_norm_rev.recipient_doing_business_str, data_norm_rev.recipient_name_str)

data_norm_rev

## Step 6: Evaluating accuracy

### Step 6a: Alternative strategies

Instead of going the route of text normalization, conversion of sentence embeddings could be implemented to allow more flexibility and fine-tuning in the fuzzy matching procedure. With every word transformed into a numeric value, the resulting list per row could be compared to other rows in terms of a threshold cosine similarity or Levenshtein distance and fuzzy matching could be performed that way. The other benefit to this tactic would the ability for the analyst to control the matching threshold value. I initially thought about pursuing this strategy instead of or in addition to the normalization pipeline I ultimately used, but I did not have enough time to think about how to build a function that would do this pairwise calculation quickly between every possible pair of supplier names in this dataset, perhaps through k-means or other naive clustering analysis (resulting category labels could be appended to original df). It would be interesting to see how the accuracy of this strategy would compare to the strategy I ultimately used (the text normalization strategy may lead to an overcount of unique suppliers, while a strategy such as this, if the hyperparameters are not well tuned, may lead to an undercount of unique suppliers).

## Step 7: Newly answerable questions